In [38]:
using Combinatorics

using NBInclude
@nbinclude("randommap.ipynb")
@nbinclude("helpers.ipynb")
# @nbinclude("testing_RM.ipynb")

In [39]:
function convert_maptodcjdist_to_converse(map_to_dedupstr_dcjdist::Dict{Vector{Int}, Tuple{String, Int}})
    dcjdist_to_map = Dict{Int, Vector{Vector{Int}}}()

    for (map, tuple) in map_to_dedupstr_dcjdist
        dcjdist = tuple[2]

        if dcjdist in keys(dcjdist_to_map)
            push!(dcjdist_to_map[dcjdist], map)
        else 
            dcjdist_to_map[dcjdist] = [map]
        end 
    end 

    return dcjdist_to_map
end 

convert_maptodcjdist_to_converse (generic function with 1 method)

In [40]:
function insert_new_map_into_documentation(neighbor::Vector{Int}, dcjdist::Int, dedup_genome::String, map_to_dedupstr_dcjdist::Dict{Vector{Int}}, dcjdist_to_map::Dict{Int, Vector{Vector{Int}}}, sorted_dists::Vector{Int})
    if neighbor in keys(map_to_dedupstr_dcjdist) 
        throw(ArgumentError("ERROR: generated a repeat map"))
    end

    map_to_dedupstr_dcjdist[neighbor] = (dedup_genome, dcjdist)
    
    if dcjdist in keys(dcjdist_to_map)
        push!(dcjdist_to_map[dcjdist], neighbor)
    else 
        dcjdist_to_map[dcjdist] = [neighbor]
    end 

    if dcjdist ∉ sorted_dists
        idx = searchsortedfirst(sorted_dists, dcjdist)
        insert!(sorted_dists, idx, dcjdist)    
    end 
end 

insert_new_map_into_documentation (generic function with 1 method)

In [41]:
# old neighbor definition (increment one element of the map by 1)
function find_old_neighbor(idxs_of_neighbors::Set{Int}, map_to_explore::Vector{Int}, S_dupgene_to_multiplicity::OrderedDict{String, Int})
    i = rand(idxs_of_neighbors)
    pop!(idxs_of_neighbors, i)

    neighbor = deepcopy(map_to_explore)
    neighbor[i] += 1    

    # mod(multiplicity!)
    multiplicity = S_dupgene_to_multiplicity[collect(keys(S_dupgene_to_multiplicity))[i]]
    if neighbor[i] > factorial(multiplicity) 
        neighbor[i] = 1 
    end 
    return neighbor
end  

find_old_neighbor (generic function with 1 method)

In [42]:
function permlexrank(n::Int, perm::Vector{Int})
    r = 1
    p = deepcopy(perm) 

    for j in 1:n
        r += (p[j]-1) * (factorial(n-j))

        for i in j+1:n
            if p[i] > p[j]
                p[i] = p[i]-1
            end 
        end 
    end 
    return r 
end 

# n = 4 
# perm = [4, 3, 2, 1]

# n = 2
# perm = [1, 2]

# permlexrank(n, perm)

permlexrank (generic function with 1 method)

In [43]:
function swap2randints_fornewrank(rank::Int, gene::String, mult::Int, details::String)
    indices = collect(1:mult)
    nthperm!(indices, rank)
    
    details = details * " || order of " * string(mult) * " dups of '" * gene * "' " * string(indices) * " (rank " * string(rank) * ")"

    # randomly choose two unique indices/ 
    rand_idx1 = rand(1:mult)
    rand_idx2 = rand(1:mult)
    while rand_idx2 == rand_idx1 
        rand_idx2 = rand(1:mult)
    end 

    # swap those two 
    tmp  = indices[rand_idx1]
    indices[rand_idx1] = indices[rand_idx2]
    indices[rand_idx2] = tmp

    new_rank = permlexrank(mult, indices)

    details = details * " --> " * string(indices) * " (rank " * string(new_rank) * ")"   # swapped 2 rand elem

    return new_rank, details

end 

swap2randints_fornewrank (generic function with 1 method)

In [44]:
function find_min_change_neighbor(map_to_explore::Vector{Int}, dupgene_to_multiplicity::OrderedDict{String, Int}, mapidx_to_gene::Dict{Int, String})
    neighbor = deepcopy(map_to_explore)

    # choose dup gene 
    i = rand(1:length(neighbor))
    rank = neighbor[i]
    gene = mapidx_to_gene[i]
    mult = dupgene_to_multiplicity[gene]

    details = string("changing map for " * gene * " (idx " * string(i) * " in map)")

    new_rank, details = swap2randints_fornewrank(rank, gene, mult, details) 
    neighbor[i] = new_rank   
    
    return neighbor, details
end 

find_min_change_neighbor (generic function with 1 method)

In [45]:
# given string S and two maps m and v, v = neighbor(m) if 
    # for a replicated gene α at idx i in the maps, v[i] = (m[i] + 1) (mod occ(α,S)!)  
    # all other genes (idx e) are mapped the same way v[e] = m[e]
function find_neighbors(map_to_explore::Vector{Int}, max_neighbors_to_explore::Int, map_to_dedupstr_dcjdist::Dict{Vector{Int}, Tuple{String, Int}}, dupgene_to_multiplicity::OrderedDict{String, Int}, mapidx_to_gene::Dict{Int, String}, min_change_neighbor::Bool, m::Int)  
    neighbors = Set{Vector{Int}}()
    
    if min_change_neighbor

        #  cap neighbors to explore 
        max_possible_minchangeneigh = 0 
        for v in values(dupgene_to_multiplicity)
            max_possible_minchangeneigh += binomial(v, 2)
        end 
        max_neighbors_to_explore = cap_val(max_neighbors_to_explore, max_possible_minchangeneigh, "neighbor", false, m) 
        seen_neighbors = Set{Vector{Int}}()
        
        # generate neighbors 
        neighbor = map_to_explore
        for i in 1:max_neighbors_to_explore
            details = ""
            while (neighbor in neighbors) || (neighbor in keys(map_to_dedupstr_dcjdist))
                if length(seen_neighbors) == max_possible_minchangeneigh
                   return collect(neighbors) 
                end 
                
                neighbor, details = find_min_change_neighbor(map_to_explore, dupgene_to_multiplicity, mapidx_to_gene)
                if neighbor ∉ seen_neighbors
                    push!(seen_neighbors, neighbor)
                end 
            end 
            if m >= 2 
                println("      found ", neighbor, " by ", details)
            end 
            push!(neighbors, neighbor)
        end 

        return collect(neighbors)
        
    else # old definition of neighbors given in paper  
        idxs_of_neighbors = Set(range(1, length(map_to_explore)))  # index of the dup gene in a map that's incremented 
    
        # cap neighbors to explore
        max_neighbors_to_explore = cap_val(max_neighbors_to_explore, length(map_to_explore), "neighbor", false, m) 

        # generate neighbors 
        neighbor = map_to_explore
        for i in 1:max_neighbors_to_explore
            while (neighbor in neighbors) || (neighbor in keys(map_to_dedupstr_dcjdist))
                if length(idxs_of_neighbors) == 0
                    return collect(neighbors)
                end
                neighbor = find_old_neighbor(idxs_of_neighbors, map_to_explore, dupgene_to_multiplicity)
            end 

            push!(neighbors, neighbor)
        end 

        return collect(neighbors)
    end    
end 



find_neighbors (generic function with 1 method)

In [46]:
# LS helpers 

function map_setup(rand_map_info::Vector, S::String, P::String, rand_maps::Int, m::Int)
    S_arr = string_to_genomearr(S)
    P_arr = string_to_genomearr(P)
    
    # if already created a set of random maps 
    if rand_map_info == Vector()
        # create arbitrary map for P, a set of random maps S_M, rank maps using estimator algo 
        _, P_dedup, map_to_dedupstr_dcjdist, S_dupgene_to_multiplicity, mapidx_to_gene = generate_random_maps_and_calc_distances(S_arr, P_arr, rand_maps, m)
    else 
        P_dedup, map_to_dedupstr_dcjdist, S_dupgene_to_multiplicity, mapidx_to_gene = deepcopy(rand_map_info)
    end 

    return P_dedup, map_to_dedupstr_dcjdist, S_dupgene_to_multiplicity, mapidx_to_gene, S_arr
end 


function cap_maps(total_maps::Int, rand_maps::Int, max_neighbors::Int, total_possible_maps::Int, m::Int)
    total_maps = cap_val(total_maps, total_possible_maps, "total", true, m)
    rand_maps = cap_val(rand_maps, total_possible_maps, "total", false, m)
    max_neighbors = cap_val(max_neighbors, total_maps - rand_maps, "neighbor", false, m)

    return total_maps, rand_maps, max_neighbors, rand_maps
end 


function select_bestmap(sorted_dists::Vector{Int}, dcj_dist_to_map::Dict{Int, Vector{Vector{Int}}}, m::Int, num_generated_maps::Int, total_maps::Int)
    smallest_dcj_dist = sorted_dists[1]
    maps = dcj_dist_to_map[smallest_dcj_dist]
    
    map_smallestd = popfirst!(maps)
    if m >= 1
        printstyled("\ngenerated ", string(num_generated_maps) * "/" * string(total_maps), "\n", color=:green)
        println("exploring neighborhood of ", map_smallestd)
    end 
    # update documentation 
    if isempty(maps)
        delete!(dcj_dist_to_map, smallest_dcj_dist)
        popfirst!(sorted_dists)
    end 

    return smallest_dcj_dist, map_smallestd

end 

function explore_neighbors(neighbors:: Vector{Vector{Int}}, S_arr::Vector{Vector{String}}, S_dupgene_to_multiplicity::OrderedDict{String, Int}, P_dedup::String, map_to_dedupstr_dcjdist::Dict{Vector{Int}}, dcj_dist_to_map::Dict{Int, Vector{Vector{Int}}}, sorted_dists::Vector{Int}, diff_bt_neighbors::Vector{Int}, smallest_dcj_dist::Int, global_min_map::Vector{Int}, global_min_dcj::Int, global_min_dedupstr::String, m::Int)   
    i = 1
    for neighbor_map in neighbors  
        s_dedup = deduplicate_genome(S_arr, S_dupgene_to_multiplicity, neighbor_map)
        d = calculate_distance(P_dedup, s_dedup, "none")
        insert_new_map_into_documentation(neighbor_map, d, s_dedup, map_to_dedupstr_dcjdist, dcj_dist_to_map, sorted_dists)
        push!(diff_bt_neighbors, d-smallest_dcj_dist)
        if m >= 1
            println("     ", neighbor_map, " dcj dist=", d)
        end 

        if d < global_min_dcj
            global_min_map  = neighbor_map
            global_min_dcj =  map_to_dedupstr_dcjdist[global_min_map][2]
            global_min_dedupstr = map_to_dedupstr_dcjdist[global_min_map][1]

            if m >= 1
                print("    !!!!!found a min dcj mapping ", global_min_map, " with distance ", global_min_dcj, "   ")
                println(P_dedup, " --> ", global_min_dedupstr)
            end 
        end 
        i += 1
    end 

    return global_min_map, global_min_dcj, global_min_dedupstr
end 


explore_neighbors (generic function with 1 method)

In [47]:
# local search heuristic

# total_maps = total number of maps to be created 
# rand_maps = number of maps randomly generated 
# max_neighbors = max number of neighbors explored in each local search
function localsearch(S::String, P::String, total_maps::Int, rand_maps::Int, max_neighbors::Int, mode::String, min_change_neighbor::Bool, rand_map_info::Vector)
    m = setmode(mode)     

    if m >= 1
        printstyled("SRC " * S * " --> TARGET " * P * "\n", color=:cyan)
        println("total maps=", total_maps, " || num_rand_maps=", rand_maps, " || max_neighbors=", max_neighbors, "\n")
    end 

    P_dedup, map_to_dedupstr_dcjdist, S_dupgene_to_multiplicity, mapidx_to_gene, S_arr = map_setup(rand_map_info, S, P, rand_maps, m)

    # cap maps explored 
    total_possible_maps = 1
    for (_, mult) in S_dupgene_to_multiplicity
        total_possible_maps *= factorial(mult)
    end 
    total_maps, rand_maps, max_neighbors, num_generated_maps = cap_maps(total_maps, rand_maps, max_neighbors, total_possible_maps, m)
    
    # create data structures to find best map in O(1)
    dcj_dist_to_map = convert_maptodcjdist_to_converse(map_to_dedupstr_dcjdist)
    sorted_dists = sort(collect(keys(dcj_dist_to_map)))

    # initialize vars storing best map and corresponding details (returned at the end)
    smallest_dcj_dist = sorted_dists[1]
    global_min_map = dcj_dist_to_map[smallest_dcj_dist][1]
    global_min_dcj = sorted_dists[1]
    global_min_dedupstr = map_to_dedupstr_dcjdist[global_min_map][1]
    
    diff_bt_neighbors = Vector{Int}()
    
    # until 'total_maps' maps are generated
    while total_maps != num_generated_maps
        if sorted_dists == Vector{Int}() 
            println("WARNING: ran out of neighbors to explore; only explored ", num_generated_maps, "/", total_maps, " for ", rand_maps, " rand maps & ", max_neighbors, " max neighbors") 
            break
        end 

        # select best not yet explored map
        smallest_dcj_dist, map_smallestd = select_bestmap(sorted_dists, dcj_dist_to_map, m, num_generated_maps, total_maps)
        
        # cap neighbors 
        max_neighbors = cap_val(max_neighbors, total_maps - num_generated_maps, "neighbor", false, m)

        # searches up to 'max_neighbors' neighbor maps
        neighbors = find_neighbors(map_smallestd, max_neighbors, map_to_dedupstr_dcjdist, S_dupgene_to_multiplicity, mapidx_to_gene, min_change_neighbor, m)
        if m >= 1
            println("neighbors: ")
        end 

        # explore neighbors 
        global_min_map, global_min_dcj, global_min_dedupstr = explore_neighbors(neighbors, S_arr, S_dupgene_to_multiplicity, P_dedup, map_to_dedupstr_dcjdist, dcj_dist_to_map, sorted_dists, diff_bt_neighbors, smallest_dcj_dist, global_min_map, global_min_dcj, global_min_dedupstr, m)

        # track # explored maps
        num_generated_maps += length(neighbors)
    end

    return global_min_dcj, global_min_map, global_min_dedupstr, map_to_dedupstr_dcjdist, diff_bt_neighbors
end 


localsearch (generic function with 1 method)

In [49]:
# src = ".a.,a'ab,a'b'c"  
# target = "ab,a,b,c'a'a"

# total_maps = 7
# rand_maps = 3
# max_neighbors = 3

# mode = "trace"
# min_change_neighbor = true 

# global_min_dcj, global_min_map, global_min_dedupstr, map_to_dedupstr_dcjdist, differences = localsearch(src, target, total_maps, rand_maps, max_neighbors, mode, min_change_neighbor, Vector())